In [ ]:
from pathlib import Path

base_dir = Path('../../local/out/elpp/')

In [ ]:
import pandas as pd

# name = "exp3_emb_size_10_hidden_size_16_reasoners_300_concepts_5_roles_200_proofs.feather"
name = "exp3_emb_size_32_hidden_size_32_reasoners.feather"
# name = "exp3_emb_size_32_hidden_size_16_reasoners_300_concepts_5_roles_200_proofs.feather"
# name = "exp3.feather" # default values
# name = "exp3_emb_size_300_hidden_size_300_reasoners.feather"
name = "exp3_emb_size_300_hidden_size_300_reasoners_300_concepts_5_roles_200_proofs.feather"

df = pd.read_feather(base_dir / name)
tmp = df.apply(lambda x:x['Axiom'], result_type='expand', axis=1)
df = pd.merge(df, tmp, left_index=True, right_index=True).rename(columns={0: 'Operator', 1: 'Subclass', 2: 'Superclass'})
df

## Reasoner training complexity (j) and embeddings training complexity (k) vs recall on the test set >k

In [ ]:
df[['Complexity threshold j', 'Complexity threshold k', 'Predicted']].groupby(['Complexity threshold j', 'Complexity threshold k']).mean().reset_index().pivot(index='Complexity threshold j', columns='Complexity threshold k', values='Predicted')

## Reasoner training complexity (j) and embeddings training complexity (k) vs recall on the test set >20


In [ ]:
summary = df.pivot(index=['KB', 'Operator', 'Subclass', 'Superclass'], columns=['Complexity threshold k', 'Complexity threshold j'], values='Predicted').dropna().mean(axis=0).reset_index().pivot(index='Complexity threshold j', columns='Complexity threshold k', values=0)
summary

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 5))
ax = sns.heatmap(summary, annot=True, fmt=".2f", cmap='gist_gray')
ax.set_xlabel('$k$')
ax.set_ylabel('$j$')
plt.savefig(base_dir / 'exp3.pdf', dpi=300)

# KB = dataset, (j, k) = classifier

This results in $19^2\cdot (19^2-1)/2=64,980$ comparisons. Assuming family-wise error rate $0.05$ and the Bonferroni correction, the significance level would be $0.05/64980=7.69\cdot 10^{-7}$ and that clearly makes no sense whatsoever.


In [ ]:
summary = df.pivot(index=['KB', 'Operator', 'Subclass', 'Superclass'], columns=['Complexity threshold j', 'Complexity threshold k'], values='Predicted').dropna().reset_index().drop(columns=['Operator', 'Subclass', 'Superclass']).groupby('KB').mean()
summary.columns = summary.columns.to_flat_index()
summary

In [ ]:
from scipy.stats import friedmanchisquare

friedmanchisquare(*summary.to_numpy())


In [ ]:
import scikit_posthocs as sp

sp.posthoc_nemenyi_friedman(summary, melted=False) #.to_numpy())